# Notebook example for STEM simulation postprocessing
This notebook illustrates how to use HyperSpy to analyse STEM simulation results produced. The results should already be converted to a hyperspy-readable format, such as ".hspy" and preferably through the use of `mul2py`s functionality. 

## Content
  1. [Loading and inspection](#1-loading-and-inspecting-your-results)
  2. [Blurring your signal](#2-blurring-your-signal)
  3. [Making thickness profiles](3-making-thickness-profiles-of-atomic-column-scattering)
  4. [Other options (development)](4-other-options)

## 1 Loading and inspecting your results
Start by setting up matplotlib and importing required packages:

In [35]:
%matplotlib qt
import hyperspy.api as hs

Next, we load the result into a hyperspy signal:

In [36]:
signal = hs.load("STEM_results.hspy")

It is always useful to print some information about the signal and its axes:

In [37]:
print(signal.metadata)
print(signal.axes_manager)

├── General
│   ├── elapsed_time = 4430.569988000006
│   ├── original_filename = /lustre1/work/emilc/MULTEM/Test/STEMgpu_results.ecmat
│   └── title = STEMgpu_results
├── Signal
│   ├── binned = False
│   └── signal_type = 
└── SimulationParameters
    ├── E_0 = 200.0
    ├── cond_lens_c_10 = 14.0312
    ├── cond_lens_c_30 = 0.0025
    ├── cond_lens_outer_aper_ang = 27.0
    ├── detector
    │   ├── cir
    │   │   ├── inner_ang
    │   │   │   └── resolved_reference
    │   │   │       └── resolved_reference = 48.0
    │   │   └── outer_ang
    │   │       └── resolved_reference
    │   │           └── resolved_reference = 200.0
    │   ├── matrix
    │   │   ├── R = 0.0
    │   │   └── fR = 0.0
    │   ├── radial
    │   │   ├── fx = 0.0
    │   │   └── x = 0.0
    │   └── type = 1.0
    ├── nx = 2048.0
    ├── ny = 2048.0
    ├── scanning_ns = 50.0
    ├── scanning_periodic = 0.0
    ├── scanning_x0 = 16.2
    ├── scanning_xe = 28.35
    ├── scanning_y0 = 16.2
    ├── scanning_ye = 

If the file was generated by `mul2py.buildtools.builders.make_signal()`, the metadata should be useful and relevant to the simulation type, and the axes should be calibrated. Here, we can see that the simulation was performed with $ E=200 $ kV, the potentials sampling was $ 2048\times2048 $, and we can also see the specimen information. From the axes manager, we can see there are four dimensions, namely the thickness (` "z" `), the detectors (` "detector" `) of which there are two, and the $x$ and $y$ dimensions (` "x" `, and ` "y" `). 

Next, we plot the signal to take a look at these axes. 

In [28]:
signal.plot() #Plot the signal


Detector 0 is obviously a bright-field detector, while Detector 1  is an annular dark-field detector. The columns look rather pixelated even if we have used a relatively fine step size. This is because this simulation was not conducted with the spatial and temporal incoherence of the source taken into account, and therefore the beam size is not correct. This can be "postsimulated" by blurring the images with a gaussian of reasonable size. The size of the gaussian must be guessed, but it should be in the range $0.5$ Å to $\sim1$ Å for an aberration corrected microscope, and larger for other microscopes. Let us try to simulate the probe size.
## 2 Blurring your signal
First, the required packages must be imported: 

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter

Then, the size of the gaussian must be set and tuned to fit the scale of the simulation:

In [ ]:
fwhm = 1.0 #FWHM [Å]
fwhm /= signal.axes_manager['x'].scale #Make the FWHM dimensionless by scaling it to the signal scale
variance = fwhm / (2 * np.sqrt( 2 * np.log( 2 ) ) ) #Convert the FWHM to a variance

Finally, the gaussian filter is mapped to the signal space of the signal (applied to all images of all detectors and all thicknesses):

In [ ]:
blurred_signal = signal.map(gaussian_filter, inplace = False, sigma = variance, mode = 'wrap') #Map the gaussian filter to the signal

blurred_signal.plot() #Plot the signal

If the simulation results makes sense and you are satisfied with the blur, you can overwrite the original `signal` variable with the blurred signal and continue your postprocessing and analysis:

In [ ]:
signal = blurred_signal.deepcopy()

## 3 Making thickness profiles of atomic column scattering
If we want to make a thickness profile of one of these atomic columns, we must first make a region of interest and use it to extract the column from the signal:

In [30]:
roi = hs.roi.CircleROI(cx=19.07, cy=17.13, r=1.025) #Make a region of interest
print(roi)
roi.add_widget(signal, axes=['x', 'y']) #Connect the roi to the signal
cropped_signal = roi(signal) #Extract the roi from the signal (does not affect the original signal)


CircleROI(cx=19.07, cy=17.13, r=1.025)


With this cropped signal, we can integrate the signal space to take a look at the "total" scattering from this column and how it develops through the thickness (this is a more robust way of getting the famous $Z$-contrast of HAADF STEM than doing single-pixel evaluation. You could also do this with several annular detectors to see how the "total" scattering changes as a function of probe-distance from the column center - which says something about the channelling of the beam through the thickness.

In [34]:
scattering_thickness_profile = cropped_signal.sum(axis=('x', 'y')) #Sum the signal in the x and y axes

This thickness profile contains both detectors, which can make it difficult to interpret a plot generated by `scattering_thickness_profile.plot()` as the two detectors will share the colorbar. Instead, you can use HyperSpy's indexing tool `.inav[]` to select the thicknesses and detectors you want:

In [ ]:
detector = 1 #Which detector to plot the thickness profile for
scattering_thickness_profile.inav[:,detector].plot() #Plot the thickness profile for the scattering to a certain detector angle interval


The above cell makes a thickness profile for the complete thickness (because of the `:` slice passed as the first index to the `inav` object. If you only want a every other thickness for instance, you can instead call:

In [39]:
detector = 1
scattering_thickness_profile.inav[0:-1:2, detector].plot()

## 4 Other options
Other options are also possible. For instance, you can use machine learning algorithms to attempt to extract scattering behaviour as a function of thickness, beam position, and/or scattering angle interval (if more detectors are used in the simulation). This is not presently covered in this guide, but may be added if there is interest.
